# Setup and Training CNNs for Tree Genera Classification

The following Jupyter Notebook includes code to setup and train a convolutional neural network with Python and Pytorch.

Version: April 2024



In [25]:

# Imports for Pytorch
import torch # version 2.1.2
import torchvision # version 0.16.2
from torchvision.datasets import ImageFolder
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torchvision.utils import make_grid
from torchvision.transforms import v2

# Image processing and display
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.image import imread
from IPython.display import clear_output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Other Imports
import os
import shutil
import random
import numpy as np
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import pandas as pd
import time


## Setup Training and Testing Data for AutoArborist Images

Define Source and Target Directories for Training and Testing Datasets From AutoArborist Records

In [26]:
# Set the paths for Autoarborist training and testing data

# Source: contains all available street view images of tree genera from Autoarborist
source_root = r"Z:\auto_arborist_cvpr2022_v0.15\data\autoarborist_original_data\autoarborist_original_jpegs\jpegs_streetlevel_genus_idx_label"

# Target: location for images of tree genera as training data 
training_destination_root = r"Z:\auto_arborist_cvpr2022_v0.15\data\tree_classification\autoarborist\training_dataset_small_10thp_apr624"

# Target: location for images of tree genera as testing data 
testing_destination_root = r"Z:\auto_arborist_cvpr2022_v0.15\data\tree_classification\autoarborist\testing_dataset_small_10thp_apr624"

# Existing Target: location for existing images of tree genera as training data used in previous experiments
existing_training_root = r"Z:\auto_arborist_cvpr2022_v0.15\data\tree_classification\autoarborist\training_dataset_small_march624"

# Existing Target: location for existing images of tree genera as testing data used in previous experiments
existing_testing_root = r"Z:\auto_arborist_cvpr2022_v0.15\data\tree_classification\autoarborist\testing_dataset_small_march624"

# Append: logical statement to append new images to existing training and testing data
append = True

# CSV path: location to save the CSV file containing the training and testing data
csv_path = r"Z:\auto_arborist_cvpr2022_v0.15\data\tree_classification\autoarborist"


### Create a directory with folders named for each class, using the PyTorch ImageFolder Class

https://pytorch.org/vision/stable/generated/torchvision.datasets.ImageFolder.html


In [33]:
# Create a directory of images with folders named for each class, using the PyTorch ImageFolder Class
# https://pytorch.org/vision/stable/generated/torchvision.datasets.ImageFolder.html
# Each directory should contain a set of images labelled to genus

# Ratio of training to testing images
training_ratio = 0.9
testing_ratio = 0.1

# Maximum number of images for training and testing
max_training_images_reset = 2250
max_testing_images_reset = 250

# Dictionary to keep track of selected training images for each genus
selected_training_images = {}

# Selected set of genera
selected_genera = ['acer','ailanthus','betula','citrus','fraxinus','gleditsia','juglans','juniperus',
                   'magnolia','phoenix','picea','pinus','prunus','pseudotsuga','pyrus','quercus','rhus','sequoia','taxodium',
                   'thuja','tilia','ulmus','washingtonia']


# Select all genera
#selected_genera = os.listdir(source_root)

# Iterate through the source directory
for genus_folder in os.listdir(source_root):
    max_training_images = max_training_images_reset
    max_testing_images = max_testing_images_reset
    # Keep track of starting time
    start_time = time.time()
    genus_path = os.path.join(source_root, genus_folder) # Get path to images for each genera
    
    # List all images in the current genus folder. Some images are .jpg and .jpeg foramt.
    images = [image for image in os.listdir(os.path.join(genus_path, 'images')) if image.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    # Only select genera with >100 images.
    if len(images) > 100:
        # Check if it's a directory and if it's in the selected genera list
        if os.path.isdir(genus_path) and genus_folder in selected_genera:
            print(f"Processing images for {genus_folder}...")
            # Create destination folders for the training and testing data for the current genus
            training_destination_genus_path = os.path.join(training_destination_root, genus_folder)
            testing_destination_genus_path = os.path.join(testing_destination_root, genus_folder)
            os.makedirs(training_destination_genus_path, exist_ok=True)
            os.makedirs(testing_destination_genus_path, exist_ok=True)

            # If append is True, first copy existing training and testing images to the new training and testing folders
            # Update the max_training_images and max_testing_images to reflect the new number of images
            # Update images to exclude the existing training and testing images
            if append:
                print(f"Copying existing images for {genus_folder}... to new training and testing folders.")
                existing_training_genus_path = os.path.join(existing_training_root, genus_folder)
                existing_testing_genus_path = os.path.join(existing_testing_root, genus_folder)

                # Copy existing training images to the new training destination folder
                for image in os.listdir(existing_training_genus_path):
                    source_image_path = os.path.join(existing_training_genus_path, image)
                    destination_image_path = os.path.join(training_destination_genus_path, image)
                    _= shutil.copy2(source_image_path, destination_image_path)

                # Copy existing testing images to the new testing destination folder
                for image in os.listdir(existing_testing_genus_path):
                    source_image_path = os.path.join(existing_testing_genus_path, image)
                    destination_image_path = os.path.join(testing_destination_genus_path, image)
                    _= shutil.copy2(source_image_path, destination_image_path)

                # Update the max_training_images and max_testing_images

                max_training_images = max_training_images - len(os.listdir(existing_training_genus_path))
                max_testing_images = max_testing_images - len(os.listdir(existing_testing_genus_path))
                print(f"Updated max_training_images: {max_training_images}, max_testing_images: {max_testing_images}")

                # Update images to exclude the existing training and testing images
                print(f"Total number of available images: {len(images)} for {genus_folder}...")
                existing_training_images = set(os.listdir(existing_training_genus_path))
                existing_testing_images = set(os.listdir(existing_testing_genus_path))
                images = [image for image in images if image not in existing_training_images and image not in existing_testing_images]
                print(f"Total number of images after excluding existing training and testing images: {len(images)} for {genus_folder}...")

            # Randomly select a number of images from the folder here: (900 training + 100 testing).
            if len(images) > max_training_images + max_testing_images:
                images = random.sample(images, max_training_images + max_testing_images) # file paths for images

            # Randomly divide images into training and testing sets
            num_total_images = len(images)
            num_training_images_to_copy = min(int(num_total_images * training_ratio), max_training_images)
            num_testing_images_to_copy = min(num_total_images - num_training_images_to_copy, max_testing_images)

            # Randomly shuffle the images before moving
            random.shuffle(images)

            # Split images into training and testing sets
            training_images = images[:num_training_images_to_copy]
            testing_images = images[num_training_images_to_copy:]

            # Copy training images to the training destination folder
            print(f"Copying {len(training_images)} training images for {genus_folder}...")
            for image in training_images:
                source_image_path = os.path.join(genus_path, 'images', image)
                destination_image_path = os.path.join(training_destination_genus_path, image)
                _= shutil.copy2(source_image_path, destination_image_path)

            # Copy testing images to the testing destination folder
            print(f"Copying {len(testing_images)} testing images for {genus_folder}...")
            for image in testing_images:
                source_image_path = os.path.join(genus_path, 'images', image)
                destination_image_path = os.path.join(testing_destination_genus_path, image)
                _= shutil.copy2(source_image_path, destination_image_path)
            # Keep track of ending time
            end_time = time.time()
            # Report time take in minutes
            total_time = (end_time - start_time) / 60
            print(f"Images copied successfully for {genus_folder}. Time taken: {total_time} minutes.")

print(f"All images copied successfully for: {selected_genera}.")

Processing images for acer...
Copying existing images for acer... to new training and testing folders.
Updated max_training_images: 1350, max_testing_images: 150
Total number of available images: 82051 for acer...
Total number of images after excluding existing training and testing images: 81051 for acer...
Copying 1350 training images for acer...
Copying 150 testing images for acer...
Images copied successfully for acer. Time taken: 3.804543137550354 minutes.
Processing images for ailanthus...
Copying existing images for ailanthus... to new training and testing folders.
Updated max_training_images: 1350, max_testing_images: 150
Total number of available images: 3723 for ailanthus...
Total number of images after excluding existing training and testing images: 2723 for ailanthus...
Copying 1350 training images for ailanthus...
Copying 150 testing images for ailanthus...
Images copied successfully for ailanthus. Time taken: 2.947280395030975 minutes.
Processing images for betula...
Copyi

### Create CSV files (metadata) documenting image filenames in the testing and training datasets

In [12]:
# # Selected set of genera
# selected_genera = ['acer','ailanthus','betula','citrus','fraxinus','gleditsia','juglans','juniperus',
#                    'magnolia','phoenix','picea','pinus','prunus','pseudotsuga','pyrus','quercus','rhus','sequoia','taxodium',
#                    'thuja','tilia','ulmus','washingtonia']

# def process_existing_files(root_directory, file_type):
#     existing_files = {}

#     for genus in selected_genera:
#         genus_dir = os.path.join(root_directory, genus)
#         genus_files = os.listdir(genus_dir)
#         existing_files[genus] = genus_files

#     # Export as table with keys as columns
#     df = pd.DataFrame(existing_files)

#     # Write to csv
#     csv_filepath = os.path.join(csv_path, os.path.basename(root_directory) + ".csv")
#     df.to_csv(csv_filepath, index=False)

#     print(f"{file_type.capitalize()} files processed successfully. CSV file saved at {csv_path}")

# # Process existing training files
# process_existing_files(existing_training_root, "training")

# # Process existing testing files
# process_existing_files(existing_testing_root, "testing")

# # To do: This workflow will be updated in the future so that it's created after each new testing and training dataset is created.

Training files processed successfully. CSV file saved at Z:\auto_arborist_cvpr2022_v0.15\data\tree_classification\autoarborist
Testing files processed successfully. CSV file saved at Z:\auto_arborist_cvpr2022_v0.15\data\tree_classification\autoarborist


### Print the number of images in the training and testing directories

In [ ]:
# How many images are contained in the training and testing directories?

def print_directory_info(root_directory):
    for genus_folder in os.listdir(root_directory):
        genus_path = os.path.join(root_directory, genus_folder)
        
        # Check if it's a directory
        if os.path.isdir(genus_path):
            # Count the number of files in the directory
            num_files = len([f for f in os.listdir(genus_path) if os.path.isfile(os.path.join(genus_path, f))])
            
            print(f"Directory: {genus_folder}, Number of Files: {num_files}")

# Print information for the training directory
print("Training Directory Information:")
print_directory_info(training_destination_root)

# Print information for the testing directory
print("/nTesting Directory Information:")
print_directory_info(testing_destination_root)


## Setup Training and Testing Data for iNaturalist images

Define Source and Target Directories for Training and Testing Datasets From iNaturalist Records

In [ ]:

# Source: contains all available street view images of tree genera from iNaturalist
source_root = r"Z:\auto_arborist_cvpr2022_v0.15\data\tree_classification\inat\images\original_10k"

# Target: location for images of tree genera as training data 
training_destination_root = r"Z:\auto_arborist_cvpr2022_v0.15\data\tree_classification\inaturalist\training_dataset_small_10thp_apr624"

# Target: location for images of tree genera as testing data 
testing_destination_root = r"Z:\auto_arborist_cvpr2022_v0.15\data\tree_classification\inaturalist\testing_dataset_small_10thp_apr624"

# Existing Target: location for existing images of tree genera as training data used in previous experiments
existing_training_root = r"Z:\auto_arborist_cvpr2022_v0.15\data\tree_classification\inaturalist\training_dataset_small_march624"

# Existing Target: location for existing images of tree genera as testing data used in previous experiments
existing_testing_root = r"Z:\auto_arborist_cvpr2022_v0.15\data\tree_classification\inaturalist\testing_dataset_small_march624"

# Append: logical statement to append new images to existing training and testing data
append = True

### Create a directory with folders named for each class, using the PyTorch ImageFolder Class

https://pytorch.org/vision/stable/generated/torchvision.datasets.ImageFolder.html


In [ ]:
# Create a directory of images with folders named for each class, using the PyTorch ImageFolder Class: https://pytorch.org/vision/stable/generated/torchvision.datasets.ImageFolder.html
# Each directory should contain a set of images labelled to genus

# Ratio of training to testing images
training_ratio = 0.9
testing_ratio = 0.1

# Maximum number of images for training and testing
max_training_images = 13500
max_testing_images = 1500

# Dictionary to keep track of selected training images for each genus
selected_training_images = {}

# Selected set of genera
selected_genera = ['acer','ailanthus','betula','citrus','cupaniopsis','erythrina','fraxinus','gleditsia','juglans','juniperus',
                   'magnolia','phoenix','picea','pinus','prunus','pseudotsuga','pyrus','quercus','rhus','sequoia','taxodium',
                   'thuja','tilia','ulmus','washingtonia']

# Iterate through the source directory
for genus_folder in os.listdir(source_root):
    max_training_images = max_training_images_reset
    max_testing_images = max_testing_images_reset
    start_time = time.time()
    genus_path = os.path.join(source_root, genus_folder)
    
    # List all images in the current genus folder
    images = [image for image in os.listdir(genus_path) if image.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    # Only select genera with >100 images
    if len(images) > 100:
        print(f"Processing images for {genus_folder}...")
        # Check if it's a directory and if it's in the selected genera list
        if os.path.isdir(genus_path) and genus_folder in selected_genera:
            # Create destination folders for the current genus
            training_destination_genus_path = os.path.join(training_destination_root, genus_folder)
            testing_destination_genus_path = os.path.join(testing_destination_root, genus_folder)
            os.makedirs(training_destination_genus_path, exist_ok=True)
            os.makedirs(testing_destination_genus_path, exist_ok=True)

            # If append is True, first copy existing training and testing images to the new training and testing folders
            # Update the max_training_images and max_testing_images to reflect the new number of images
            # Update images to exclude the existing training and testing images
            if append:
                print(f"Copying existing images for {genus_folder}... to new training and testing folders.")
                existing_training_genus_path = os.path.join(existing_training_root, genus_folder)
                existing_testing_genus_path = os.path.join(existing_testing_root, genus_folder)

                # Copy existing training images to the new training destination folder
                for image in os.listdir(existing_training_genus_path):
                    source_image_path = os.path.join(existing_training_genus_path, image)
                    destination_image_path = os.path.join(training_destination_genus_path, image)
                    _= shutil.copy2(source_image_path, destination_image_path)

                # Copy existing testing images to the new testing destination folder
                for image in os.listdir(existing_testing_genus_path):
                    source_image_path = os.path.join(existing_testing_genus_path, image)
                    destination_image_path = os.path.join(testing_destination_genus_path, image)
                    _= shutil.copy2(source_image_path, destination_image_path)

                # Update the max_training_images and max_testing_images
                max_training_images = max_training_images - len(os.listdir(existing_training_genus_path))
                max_testing_images = max_testing_images - len(os.listdir(existing_testing_genus_path))
                print(f"Updated max_training_images: {max_training_images}, max_testing_images: {max_testing_images}")

                # Update images to exclude the existing training and testing images
                print(f"Total number of available images: {len(images)} for {genus_folder}...")
                existing_training_images = set(os.listdir(existing_training_genus_path))
                existing_testing_images = set(os.listdir(existing_testing_genus_path))
                images = [image for image in images if image not in existing_training_images and image not in existing_testing_images]
                print(f"Total number of images after excluding existing training and testing images: {len(images)} for {genus_folder}...")

            # Limit the number of images if it exceeds the maximum
            if len(images) >= max_training_images + max_testing_images:
                images = random.sample(images, max_training_images + max_testing_images)

            # Randomly select images for training and testing
            num_total_images = len(images)
            num_training_images_to_copy = min(int(num_total_images * training_ratio), max_training_images)
            num_testing_images_to_copy = min(num_total_images - num_training_images_to_copy, max_testing_images)

            # Randomly shuffle the images
            random.shuffle(images)

            # Split images into training and testing sets
            training_images = images[:num_training_images_to_copy]
            testing_images = images[num_training_images_to_copy:]

            # Copy training images to the training destination folder
            print(f"Copying {len(training_images)} training images for {genus_folder}...")
            for image in training_images:
                source_image_path = os.path.join(genus_path, image)
                destination_image_path = os.path.join(training_destination_genus_path, image)
                _= shutil.copy2(source_image_path, destination_image_path)

            # Copy testing images to the testing destination folder
            print(f"Copying {len(testing_images)} testing images for {genus_folder}...")
            for image in testing_images:
                source_image_path = os.path.join(genus_path, image)
                destination_image_path = os.path.join(testing_destination_genus_path, image)
                _= shutil.copy2(source_image_path, destination_image_path)
            end_time = time.time()
            total_time = (end_time - start_time) / 60
            print(f"Images copied successfully for {genus_folder}. Time taken: {total_time} minutes.")

print("Images copied successfully.")

### Create CSV files (metadata) documenting image filenames in the testing and training datasets

In [ ]:
# # Selected set of genera
# selected_genera = ['acer','ailanthus','betula','citrus','fraxinus','gleditsia','juglans','juniperus',
#                    'magnolia','phoenix','picea','pinus','prunus','pseudotsuga','pyrus','quercus','rhus','sequoia','taxodium',
#                    'thuja','tilia','ulmus','washingtonia']

# def process_existing_files(root_directory, file_type):
#     existing_files = {}

#     for genus in selected_genera:
#         genus_dir = os.path.join(root_directory, genus)
#         genus_files = os.listdir(genus_dir)
#         existing_files[genus] = genus_files

#     # Export as table with keys as columns
#     df = pd.DataFrame(existing_files)

#     # Write to csv
#     csv_filepath = os.path.join(csv_path, os.path.basename(root_directory) + ".csv")
#     df.to_csv(csv_filepath, index=False)

#     print(f"{file_type.capitalize()} files processed successfully. CSV file saved at {csv_path}")

# # Process existing training files
# process_existing_files(existing_training_root, "training")

# # Process existing testing files
# process_existing_files(existing_testing_root, "testing")

# # To do: This workflow will be updated in the future so that it's created after each new testing and training dataset is created.

Training files processed successfully. CSV file saved at Z:\auto_arborist_cvpr2022_v0.15\data\tree_classification\autoarborist
Testing files processed successfully. CSV file saved at Z:\auto_arborist_cvpr2022_v0.15\data\tree_classification\autoarborist


### Print the number of images in the training and testing directories

In [ ]:
# How many images are contained in the training and testing directories?

def print_directory_info(root_directory):
    for genus_folder in os.listdir(root_directory):
        genus_path = os.path.join(root_directory, genus_folder)
        
        # Check if it's a directory
        if os.path.isdir(genus_path):
            # Count the number of files in the directory
            num_files = len([f for f in os.listdir(genus_path) if os.path.isfile(os.path.join(genus_path, f))])
            
            print(f"Directory: {genus_folder}, Number of Files: {num_files}")

# Print information for the training directory
print("Training Directory Information:")
print_directory_info(training_destination_root)

# Print information for the testing directory
print("/nTesting Directory Information:")
print_directory_info(testing_destination_root)


In [ ]:
# How many images are contained in the training and testing directories?

def print_directory_info(root_directory):
    for genus_folder in os.listdir(root_directory):
        genus_path = os.path.join(root_directory, genus_folder)
        
        # Check if it's a directory
        if os.path.isdir(genus_path):
            # Count the number of files in the directory
            num_files = len([f for f in os.listdir(genus_path) if os.path.isfile(os.path.join(genus_path, f))])
            
            print(f"Directory: {genus_folder}, Number of Files: {num_files}")

# Print information for the training directory
print("Training Directory Information:")
print_directory_info(training_destination_root)

# Print information for the testing directory
print("/nTesting Directory Information:")
print_directory_info(testing_destination_root)


# Combine AutoArborist and iNatuarlist Data

Create Training and Testing Records from the Combined Autoarborist + iNaturalist Datasets


In [ ]:
import os
import shutil

def combine_image_data(source_dirs, combined_dir):
    for source_dir in source_dirs:
        for dataset_type in os.listdir(source_dir):
            dataset_type_dir = os.path.join(source_dir, dataset_type)
            combined_dataset_type_dir = os.path.join(combined_dir, dataset_type)
            os.makedirs(combined_dataset_type_dir, exist_ok=True)
            for genus in os.listdir(dataset_type_dir):
                genus_dir = os.path.join(dataset_type_dir, genus)
                combined_genus_dir = os.path.join(combined_dataset_type_dir, genus)
                os.makedirs(combined_genus_dir, exist_ok=True)
                for filename in os.listdir(genus_dir):
                    if filename.lower().endswith(('.jpg', '.jpeg', '.JPG')):
                        src_file = os.path.join(genus_dir, filename)
                        dst_file = os.path.join(combined_genus_dir, filename)
                        shutil.copyfile(src_file, dst_file)

# Source directories
source_dirs = [
    r'Z:\auto_arborist_cvpr2022_v0.15\data\tree_classification\inaturalist',
    r'Z:\auto_arborist_cvpr2022_v0.15\data\tree_classification\autoarborist'
]

# Combined directory
combined_dir = r'Z:\auto_arborist_cvpr2022_v0.15\data\tree_classification\autoarborist_inaturalist_combined'

# Combine image data
combine_image_data(source_dirs, combined_dir)


In [ ]:
# How many images are contained in the training and testing directories?

training_destination_root = r"Z:\auto_arborist_cvpr2022_v0.15\data\tree_classification\autoarborist_inaturalist_combined\training_dataset_small_april624"
testing_destination_root = r"Z:\auto_arborist_cvpr2022_v0.15\data\tree_classification\autoarborist_inaturalist_combined\testing_dataset_small_march624"

def print_directory_info(root_directory):
    for genus_folder in os.listdir(root_directory):
        genus_path = os.path.join(root_directory, genus_folder)
        
        # Check if it's a directory
        if os.path.isdir(genus_path):
            # Count the number of files in the directory
            num_files = len([f for f in os.listdir(genus_path) if os.path.isfile(os.path.join(genus_path, f))])
            
            print(f"Directory: {genus_folder}, Number of Files: {num_files}")

# Print information for the training directory
print("Training Directory Information:")
print_directory_info(training_destination_root)

# Print information for the testing directory
print("/nTesting Directory Information:")
print_directory_info(testing_destination_root)


# Define Image Augmentations

In [ ]:

# First, define the training and testing datasets (AutoArborist, iNaturalist, or combined) by specifying the file paths.
training_destination_root = r"Z:\auto_arborist_cvpr2022_v0.15\data\tree_classification\autoarborist_inaturalist_combined\training_dataset_small_march624"
testing_destination_root = r"Z:\auto_arborist_cvpr2022_v0.15\data\tree_classification\autoarborist_inaturalist_combined\testing_dataset_small_march624"

# Use Pytorch ImageFolder class to prepare training and testing datasets
train_data_dir = training_destination_root
test_data_dir = testing_destination_root

# Load the training and testing datasets as Pytorch Dataset Classes: https://pytorch.org/docs/stable/data.html
# The Pytorch torchvision.transforms module provides preprocessing functions: https://pytorch.org/vision/stable/transforms.html

train_transforms = v2.Compose([
    v2.ToImage(),
    v2.RandomResizedCrop(size=(512, 512), antialias=True),
    v2.RandomHorizontalFlip(p=0.5),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

# Ensure images are resized during testing as same dimension for training
test_transforms = v2.Compose([
    v2.ToImage(),
    v2.Resize(size=(512, 512), antialias=True),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

train_dataset = ImageFolder(train_data_dir, transform = train_transforms)
test_dataset = ImageFolder(test_data_dir, transform = test_transforms)

# Examine the train_dataset object
train_dataset

# Examine image dimensions: (3 channels, height 64, width 64)
img, label = train_dataset[0]
print(img.shape,label)


In [ ]:
# How many classes are in the training and testing datasets?
print("Classes in the Training Dataset : /n", len(train_dataset.classes))
print("Classes in the Testing Dataset : /n", len(test_dataset.classes))

In [ ]:
# Display single image

In [ ]:
# Visualize sample image in the training dataset

def display_img(img,label):
    print(f"Label : {train_dataset.classes[label]}")
    plt.imshow(img.permute(1,2,0)) #reshape image from (3, H, W) to (H, W, 3)

# Display the first image in the dataset
display_img(*train_dataset[2])



# Define Training, Validation, and Testing DataLoaders

In [ ]:
# Split training data into a validation set, and prepare dataset for training

# Define batch size for training 
bs = 32

# Define number of images for validation (typically, 10% of the training set)
val_size = 2000
train_size = len(train_dataset) - val_size

# Randomly split training data into train_data and val_data sets
train_data, val_data = random_split(train_dataset, [train_size, val_size])

print(f"Length of Train Data : {len(train_data)}") # Length of Train Data : 20000
print(f"Length of Validation Data : {len(val_data)}") # Length of Validation Data : 2000

# Use Pytorch DataLoader Class to iterate over a dataset for training: https://pytorch.org/docs/stable/data

train_dl = DataLoader(dataset = train_data, batch_size = bs, shuffle = True, num_workers = 4, pin_memory = True)
val_dl = DataLoader(dataset = val_data, batch_size = bs*2, num_workers = 4, pin_memory = True)
test_dl = DataLoader(dataset = test_dataset, batch_size = 1, num_workers = 4, pin_memory = True)


In [ ]:
# Display one batch of images

In [ ]:
# Visualize a single batch of images

def show_batch(dl):
    """Plot images grid of single batch"""
    for images, labels in dl:
        fig,ax = plt.subplots(figsize = (16,12))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(make_grid(images,nrow=16).permute(1,2,0))
        break
        
show_batch(train_dl)

# Prepare the Image Classification Model

The ImageClassificationBase Class inherits functionality from the nn.Module Class in Pytorch: https://pytorch.org/docs/stable/generated/torch.nn.Module.html

In [ ]:
# Prepare a Basic Model for Image Classification

import torch.nn as nn # contains base class for all neural network modules
import torch.nn.functional as F #https://pytorch.org/docs/stable/nn.functional.html contains common functions for training NNs (convolutions, losses, etc..)

class ImageClassificationBase(nn.Module): # https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module
    # Define a base class with functionality for model training, validation, and evaluation per epoch
    
    def training_step(self, batch):
        images, labels = batch
        out = self(images) # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch
        out = self(images) # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        acc = accuracy(out, labels) # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))
        

# Create an EfficientNetV2 Model

https://arxiv.org/abs/2104.00298

In [ ]:
# # Define a CNN Model using EfficientNetV2-S

class EfficientNetImageClassification(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        # Load the pre-trained EfficientNetV2-L Model
        self.network = torchvision.models.efficientnet_v2_s(pretrained=True)
        # Modify the final fully connected layer to match the number of classes in your dataset
        num_classes = len(train_dataset.classes)
        in_features = self.network.classifier[1].in_features
        self.network.classifier = nn.Linear(in_features, num_classes)

    def forward(self, xb):
        return self.network(xb)



In [ ]:
# # Instantiate the model
model = EfficientNetImageClassification()


In [ ]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(f'Number of Model Parameters: ', params)

# Define GPU Device and Load Data to GPU

In [ ]:
# Helper function and class to load data to GPU

def get_default_device():
    """ Set Device to GPU or CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    

def to_device(data, device):
    "Move data to the device"
    if isinstance(data,(list,tuple)):
        return [to_device(x,device) for x in data]
    return data.to(device,non_blocking = True)

class DeviceDataLoader():
    """ Wrap a dataloader to move data to a device """
    
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    
    def __iter__(self):
        """ Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b,self.device)
            
    def __len__(self):
        """ Number of batches """
        return len(self.dl)
    


In [ ]:
# Get GPU Device
device = get_default_device()
device

# Load data to GPU
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
to_device(model, device)

# Define CNN Model Fit and Evaluation Functions

In [ ]:
# Define Fit and Evaluation methods

# Do not compute new gradients when evaluating a model
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

# Fit model
def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    
    history = []
    # Create optimizer with initial learning rate
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        model.train()
        train_losses = []
        
        for batch in train_loader:
            # Forward pass: prediction & calculate loss
            loss = model.training_step(batch)
            train_losses.append(loss)
            # Backward pass: backpropagate loss & calculate gradients
            loss.backward()
            optimizer.step() #update gradients
            optimizer.zero_grad() #zero gradients for next training forward pass
            
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
        
    return history


# Load model to device and specify training parameters

- Epochs = 10
- Optimizer (Gradient Descent): Adam
- Learning Rate: 0.001


In [ ]:
# Load the model to the device
model = to_device(EfficientNetImageClassification(), device)

In [ ]:

# Set number of epochs, optimizer function, learning rate, and warmup epochs
num_epochs = 10
opt_func = torch.optim.Adam
base_lr = 0.001


# Fit Model

In [ ]:
# Fit model with warmup, logit adjustment, and record results after each epoch

# Fit model and record result after epoch
history = fit(num_epochs, base_lr, model, train_dl, val_dl, opt_func)



# Capture Fitted Model Results

In [ ]:
# Model history contains training loss, validation loss, and validation accuracy metrics
history

In [ ]:
# Save the model weights file to path
model_path = r'C:\Users\talake2\Desktop\tree-classification-autoarb_inat-25-genera-1000imgs-effnet2s-10epochs-lr001-aug-march724.pth'

# Torch.Save model to file: https://pytorch.org/tutorials/beginner/saving_loading_models.html
torch.save(model.state_dict(), model_path)

### Plot Accuracy and Loss curves

In [ ]:
def plot_accuracies(history):
    """ Plot the history of accuracies"""
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');
    

plot_accuracies(history)


In [ ]:
def plot_losses(history):
    """ Plot the losses in each epoch"""
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');

plot_losses(history)

# Create confusion matrix

In [ ]:
# Model Evaluation with Confusion Matrix
# Iterate over test dataset and generate predictions for a confusion matrix

selected_genera = ['acer','ailanthus','betula','citrus','cupaniopsis','erythrina','fraxinus','gleditsia','juglans','juniperus',
                   'magnolia','phoenix','picea','pinus','prunus','pseudotsuga','pyrus','quercus','rhus','sequoia','taxodium',
                   'thuja','tilia','ulmus','washingtonia']



y_pred = []
y_true = []

#model.cuda()  # Move model to CUDA if not already there

model.eval()  # Set the model to evaluation mode.

with torch.no_grad():
    for inputs, labels in test_dl:
        inputs, labels = inputs.cuda(), labels.cuda()
        output = model(inputs)
        output = torch.argmax(output, dim=1).cpu().numpy()  # Extract predicted labels directly
        y_pred.extend(output)
        
        labels = labels.cpu().numpy()
        y_true.extend(labels)

# Confusion Matrix
cf_matrix = confusion_matrix(y_true, y_pred)



In [ ]:

# Plot Confusion Matrix
disp = ConfusionMatrixDisplay(cf_matrix, display_labels=selected_genera)
fig, ax = plt.subplots(figsize=(10, 10))
disp.plot(cmap=plt.cm.Blues, xticks_rotation='vertical', ax=ax)
plt.savefig(r"C:\Users\talake2\Desktop\auto_arborist_cvpr2022_v015\pytorch_cnn_classifier_experiments_jan24\autoarborist_inaturalist_models_march2024\tree-classification-inat-25-genera-1000imgs-effnet2s-10epochs-lr001-march724.png", dpi=200)
#plt.close()
plt.show()


# Calculate precision, recall, F1, and support per class on the withheld testing dataset

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred, target_names = selected_genera))

# Load a trained Image Classification Model (CNN) to run predictions and calculate summary statistics

In [ ]:
# Imports for Pytorch
import torch # version 2.1.2
import torchvision # version 0.16.2
#from torchvision import transforms # https://pytorch.org/vision/stable/transforms.html # To do: implement Pytorch transforms v2 (faster, more functionality)
from torchvision.datasets import ImageFolder
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torchvision.utils import make_grid
from torchvision.transforms import v2

# Image Libraries
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.image import imread
from IPython.display import clear_output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# General Imports
import os
import shutil
import random
import numpy as np
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay




# Define Class for Image Classification

In [ ]:
# Prepare a Basic Model for Image Classification

import torch.nn as nn # contains base class for all neural network modules
import torch.nn.functional as F #https://pytorch.org/docs/stable/nn.functional.html contains common functions for training NNs (convolutions, losses, etc..)

class ImageClassificationBase(nn.Module): # https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module
    # Define a base class with functionality for model training, validation, and evaluation per epoch
    
    def training_step(self, batch):
        images, labels = batch
        out = self(images) # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch
        out = self(images) # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        acc = accuracy(out, labels) # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))
        

def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))


# Define CNN Architecture

In [ ]:
# Define a CNN Model using ResNet50
class ResNet50ImageClassification(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        # Load the pre-trained ResNet model
        self.network = torchvision.models.resnet50(weights='ResNet50_Weights.IMAGENET1K_V2')
        # Modify the final fully connected layer to match the number of classes in your dataset
        num_classes = 25 # Set number of output classes
        in_features = self.network.fc.in_features
        self.network.fc = nn.Linear(in_features, num_classes)

    def forward(self, xb):
        return self.network(xb)
    
    
# # Define a CNN Model using EfficientNetV2-S
class EfficientNetImageClassification(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        # Load the pre-trained EfficientNetV2-L Model
        self.network = torchvision.models.efficientnet_v2_s(pretrained=True)
        # Modify the final fully connected layer to match the number of classes in your dataset
        num_classes = 25 # Set number of output classes
        in_features = self.network.classifier[1].in_features
        self.network.classifier = nn.Linear(in_features, num_classes)

    def forward(self, xb):
        return self.network(xb)


# Load models and data

In [ ]:
# Helper function and class to load data to GPU

def get_default_device():
    """ Set Device to GPU or CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    

def to_device(data, device):
    "Move data to the device"
    if isinstance(data,(list,tuple)):
        return [to_device(x,device) for x in data]
    return data.to(device,non_blocking = True)

class DeviceDataLoader():
    """ Wrap a dataloader to move data to a device """
    
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
    
    def __iter__(self):
        """ Yield a batch of data after moving it to device"""
        for b in self.dl:
            yield to_device(b,self.device)
            
    def __len__(self):
        """ Number of batches """
        return len(self.dl)
    

# Create training and testing datasets

In [ ]:

training_destination_root = r"C:/Users/talake2/Desktop/auto_arborist_cvpr2022_v015/pytorch_cnn_classifier_experiments_jan24/datasets/autoarborist/training_dataset_small_march624"
testing_destination_root = r"C:/Users/talake2/Desktop/auto_arborist_cvpr2022_v015/pytorch_cnn_classifier_experiments_jan24/datasets/autoarborist/testing_dataset_small_march624"

# Use Pytorch ImageFolder class to prepare training and testing datasets
train_data_dir = training_destination_root
test_data_dir = testing_destination_root

# Load the training and testing datasets as Pytorch Dataset Classes: https://pytorch.org/docs/stable/data.html
# The Pytorch torchvision.transforms module provides preprocessing functions: https://pytorch.org/vision/stable/transforms.html

train_transforms = v2.Compose([
    v2.ToImage(),
    v2.Resize(size=(512, 512), antialias=True),
    v2.RandomHorizontalFlip(p=0.5),
    v2.ToDtype(torch.float32, scale=True)
])

# Ensure images are resized during testing as same dimension for training
test_transforms = v2.Compose([
    v2.ToImage(),
    v2.Resize(size=(512, 512), antialias=True),
    v2.ToDtype(torch.float32, scale=True),
])

train_dataset = ImageFolder(train_data_dir, transform = train_transforms)
test_dataset = ImageFolder(test_data_dir, transform = test_transforms)

# Examine the train_dataset object
train_dataset

# Examine image dimensions: (3 channels, height 64, width 64)
img, label = train_dataset[0]
print(img.shape,label)


In [ ]:
# Split training data into a validation set, and prepare dataset for training

len(train_dataset.classes)
len(test_dataset.classes)

# Define batch size for training 
bs = 32

# Define number of images for validation
val_size = 2000
train_size = len(train_dataset) - val_size

# Randomly split training data into train_data and val_data sets
train_data, val_data = random_split(train_dataset, [train_size, val_size])

print(f"Length of Train Data : {len(train_data)}") # Length of Train Data : 17212
print(f"Length of Validation Data : {len(val_data)}") # Length of Validation Data : 2000

# Use Pytorch DataLoader Class to iterate over a dataset for training: https://pytorch.org/docs/stable/data

train_dl = DataLoader(dataset = train_data, batch_size = bs, shuffle = True, num_workers = 4, pin_memory = True)
val_dl = DataLoader(dataset = val_data, batch_size = bs*2, num_workers = 4, pin_memory = True)
test_dl = DataLoader(dataset = test_dataset, batch_size = 1, num_workers = 4, pin_memory = True)


In [ ]:
# Load a pre-trained model
model_path = r'C:\Users\talake2\Desktop\\tree-classification-inat-25-genera-1000imgs-effnet2s-10epochs-lr001-march724.pth'

# Get GPU Device
device = get_default_device()
device

# Instantiate the model with the same architecture as the model which parameters you saved
model = EfficientNetImageClassification()

#load the model to the device
model = to_device(EfficientNetImageClassification(), device)

model.load_state_dict(torch.load(model_path))

model.eval() # Call model.eval() to set dropout and batch normalization layers to evaluation mode before running inference. Failing to do this will yield inconsistent inference results.
 

# Load data to GPU

In [ ]:
# Load data to GPU
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
to_device(model, device)

# Predict on testing dataset

In [ ]:
# Model Evaluation with Confusion Matrix
# Iterate over test dataset and generate predictions for a confusion matrix

import torch
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


selected_genera = ['acer','ailanthus','betula','citrus','cupaniopsis','erythrina','fraxinus','gleditsia','juglans','juniperus',
                   'magnolia','phoenix','picea','pinus','prunus','pseudotsuga','pyrus','quercus','rhus','sequoia','taxodium',
                   'thuja','tilia','ulmus','washingtonia']



y_pred = []
y_true = []

#model.cuda()  # Move model to CUDA if not already there

model.eval()  # Set the model to evaluation mode

with torch.no_grad():
    for inputs, labels in test_dl:
        inputs, labels = inputs.cuda(), labels.cuda()
        output = model(inputs)
        output = torch.argmax(output, dim=1).cpu().numpy()  # Extract predicted labels directly
        y_pred.extend(output)
        
        labels = labels.cpu().numpy()
        y_true.extend(labels)

# Confusion Matrix
cf_matrix = confusion_matrix(y_true, y_pred)



# Create confusion matrix and summary statistics for classification

In [ ]:


# Plot Confusion Matrix
disp = ConfusionMatrixDisplay(cf_matrix, display_labels=selected_genera)
fig, ax = plt.subplots(figsize=(10, 10))
disp.plot(cmap=plt.cm.Blues, xticks_rotation='vertical', ax=ax)
plt.savefig(r"C:\Users\talake2\Desktop\inat_cnn_effnet2s_evalon_autoarb_25_genera_march72024", dpi=200)
plt.close()
plt.show()


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred, target_names = selected_genera))

# Run classification on a single image

In [ ]:
import cv2

# Constant for classes    
classes = ('acer', 'fraxinus', 'quercus', 'ulmus', 'prunus', 'tilia', 'gleditsia', 'malus', 'platanus', 'liquidambar', 'pinus', 'ginkgo', 'zelkova', 'celtis', 'crataegus', 'populus', 'carpinus', 'syringa', 'lagerstroemia', 'betula')

img_path = r'C:/Users/talake2/Desktop/auto_arborist_cvpr2022_v015/auto_arborist_jpegs/jpegs_aerial_streetlevel_raw/all_cities_streetview/train/acer/streetlevel_4_7.jpg'
img = imread(img_path)
img.shape

# Resize the image
img = cv2.resize(img, (512, 512))

# Convert image to tensor using torchvision.transforms.ToTensor()
transform = transforms.ToTensor()
img = transform(img) #Transforms image to [0-1] and channels first
img = img.unsqueeze(0) #add first dimension of batch size
img = img.cuda() #push to GPU

#prdict image label
output = model(img)

# Get the index of the maximum value in the output tensor
predicted_class_index = torch.argmax(output)

# Use the index to get the corresponding class label
predicted_class_label = classes[predicted_class_index.item()]

print("Predicted Class Label:", predicted_class_label)

# Get the top 5 class predictions
top5_probabilities, top5_indices = torch.topk(output, 5)

# Convert indices to class labels
top5_class_labels = [classes[i] for i in top5_indices.squeeze().tolist()]

print("Top 5 Predicted Class Labels:", top5_class_labels)
print("Top 5 Predicted Probabilities:", top5_probabilities)